In [1]:
# https://www.kaggle.com/aherparesh/sentiment-analysis-on-amazon-product-rnn-97-acc
# https://www.kaggle.com/lele1995/amazon-reviews-sentiment-analysis

In [155]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import boto3
import io
import time, datetime
from io import StringIO
from io import BytesIO
import json

from sklearn.utils import shuffle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vimald\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vimald\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vimald\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [156]:
bucket = 'swire-datalake-dev-bucket'
inference_data_key = 'ml/training_data/Amazon_after_eda_for_training.csv'


def read_csv_file(bucket_name,inference_data_key): #,aws_access_id,aws_access_key
    """ Read the csv predcition file from s3 using boto3 client
    """
    client = boto3.client('s3') #, aws_access_key_id = aws_access_id , aws_secret_access_key= aws_access_key

    csv_obj = client.get_object(Bucket=bucket_name, Key=inference_data_key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')

    Raw_data = pd.read_csv(StringIO(csv_string))
    
    return Raw_data
from_eda = read_csv_file(bucket,inference_data_key)
from_eda =from_eda.drop('Unnamed: 0',axis=1)
from_eda

,id,reviews.text,reviews.rating
0,AVqkIhwDv8e3D1O-lebb,This product so far has not disappointed. My c...,5.0
1,AVqkIhwDv8e3D1O-lebb,great for beginner or experienced person. Boug...,5.0
2,AVqkIhwDv8e3D1O-lebb,Inexpensive tablet for him to use and learn on...,5.0
3,AVqkIhwDv8e3D1O-lebb,I've had my Fire HD 8 two weeks now and I love...,4.0
4,AVqkIhwDv8e3D1O-lebb,I bought this for my grand daughter when she c...,5.0
...,...,...,...
37722,AVqkIdZiv8e3D1O-leaJ,The battery is having more and more trouble ho...,2.0
37723,AVqkIdZiv8e3D1O-leaJ,My daughter has had this tablet for almost 2 m...,3.0
37724,AVqkIdZiv8e3D1O-leaJ,Very cheap and was not impressed at all never ...,1.0
37725,AVqkIdZiv8e3D1O-leaJ,"Hard to use, Lots of ads, and Randomly closes ...",2.0


In [158]:
from_eda = shuffle(from_eda)
from_eda = shuffle(from_eda)
from_eda = shuffle(from_eda)

In [159]:
training_data = from_eda.iloc[:30000,:]
test_data =  from_eda.iloc[30000:,:]

In [163]:
test_data['reviews.rating'].value_counts()

5.0    4938
4.0    1754
3.0     556
1.0     273
2.0     206
Name: reviews.rating, dtype: int64

## Data for testing purpose

In [165]:
'ml/training_data/Amazon_after_eda_for_training.csv'
def write_to_s3(bucket_name,result_key, raw_data): # ,aws_access_id,aws_access_key
    csv_buffer = StringIO()
    raw_data.to_csv(csv_buffer)
    resource = boto3.resource('s3') # , aws_access_key_id= aws_access_id ,aws_secret_access_key=aws_access_key
    
    return resource.Object(bucket_name,result_key).put(Body=csv_buffer.getvalue())


training_data_key = 'ml/training_data/training_data_sentiment_analysis.csv'
test_data_key = 'ml/training_data/test_data_sentiment_analysis.csv'

write_to_s3(bucket,training_data_key, training_data)
write_to_s3(bucket,test_data_key, test_data)

{'ResponseMetadata': {'RequestId': '48A2D76F600029B1',
  'HostId': '2hV4Pf2BWGireVGeWWvMZqsOZQpA2zB6oK+TYpV6NGaxIifSm6z673yJ0Mj4wkewfIsuwdXV1lY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '2hV4Pf2BWGireVGeWWvMZqsOZQpA2zB6oK+TYpV6NGaxIifSm6z673yJ0Mj4wkewfIsuwdXV1lY=',
   'x-amz-request-id': '48A2D76F600029B1',
   'date': 'Wed, 30 Sep 2020 09:27:36 GMT',
   'x-amz-version-id': 'w1bgzy5LWduBXkFFN6adg2dYJriGWlBM',
   'etag': '"1a5a15ca1a66ffa0f71b499e496e3e98"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'ETag': '"1a5a15ca1a66ffa0f71b499e496e3e98"',
 'VersionId': 'w1bgzy5LWduBXkFFN6adg2dYJriGWlBM'}

In [166]:

#training_data.to_csv('training_data_sentiment_analysis.csv')
#test_data.to_csv('test_data_sentiment_analysis.csv')

# Text Preprocessing

In [170]:
inference_data_key = 'ml/training_data/training_data_sentiment_analysis.csv'
def read_csv_file(bucket_name,inference_data_key): #,aws_access_id,aws_access_key
    """ Read the csv predcition file from s3 using boto3 client
    """
    client = boto3.client('s3') #, aws_access_key_id = aws_access_id , aws_secret_access_key= aws_access_key

    csv_obj = client.get_object(Bucket=bucket_name, Key=inference_data_key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')

    Raw_data = pd.read_csv(StringIO(csv_string))
    
    return Raw_data
final = read_csv_file(bucket,inference_data_key)
final = final.drop('Unnamed: 0',axis=1)
final.head()

,id,reviews.text,reviews.rating
0,AVphgVaX1cnluZ0-DR74,"Couldn't ask for much more, performs smoothly,...",4.0
1,AVpfl8cLLJeJML43AE3S,Ask Alexa ANYTHING and she'll search the web. ...,5.0
2,AVpfl8cLLJeJML43AE3S,"Having the echo is great, especially for playi...",5.0
3,AVphgVaX1cnluZ0-DR74,Bought this tab for the $40 dollar sale price ...,3.0
4,AVphgVaX1cnluZ0-DR74,This tablet is a great back up tablet for the ...,5.0


In [171]:
#lower case all text
final["reviews.text"]=final["reviews.text"].str.lower() 

#tokenization of words
final['reviews.text'] = final.apply(lambda row: word_tokenize(row['reviews.text']), axis=1) 

#only alphanumerical values
final["reviews.text"] = final['reviews.text'].apply(lambda x: [item for item in x if item.isalpha()]) 

#lemmatazing words
final['reviews.text'] = final['reviews.text'].apply(lambda x : [WordNetLemmatizer().lemmatize(y) for y in x])

#removing useless words
stop = stopwords.words('english')
final['reviews.text'] = final['reviews.text'].apply(lambda x: [item for item in x if item not in stop])

In [172]:
final["reviews.text"] = final["reviews.text"].apply(lambda x: str(' '.join(x))) #joining all tokens

In [173]:
final.head()

,id,reviews.text,reviews.rating
0,AVphgVaX1cnluZ0-DR74,could ask much performs smoothly expandable me...,4.0
1,AVpfl8cLLJeJML43AE3S,ask alexa anything search web love able contro...,5.0
2,AVpfl8cLLJeJML43AE3S,echo great especially playing music searching ...,5.0
3,AVphgVaX1cnluZ0-DR74,bought tab dollar sale price nice little syste...,3.0
4,AVphgVaX1cnluZ0-DR74,tablet great back tablet money,5.0


In [174]:
sentiment = {1: 0,
            2: 0,
            3: 0,
            4: 1,
            5: 1}

final["sentiment"] = final["reviews.rating"].map(sentiment)

In [175]:
len(final[final["sentiment"]==0]),len(final[final["sentiment"]==1])

(4377, 25623)

In [176]:
# https://stackoverflow.com/questions/15361885/linking-text-feature-names-to-their-tfidf-value
# building tfidf matrix to train models 
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer =TfidfVectorizer(max_df=0.9)
# text = vectorizer.fit_transform(final["reviews.text"])

In [178]:
from sklearn.feature_extraction.text import TfidfVectorizer

# tf-idf based vectors
tf = TfidfVectorizer(max_df=0.9)

# Fit the model
tf_transformer = tf.fit(final["reviews.text"])

# Dump the file
pickle.dump(tf_transformer, open("trained_tfidf_vector.pkl", "wb"))
tf_transformer

TfidfVectorizer(max_df=0.9)

In [179]:
text_new =  tf_transformer.transform(final["reviews.text"])
text_new

<30000x11126 sparse matrix of type '<class 'numpy.float64'>'
	with 420812 stored elements in Compressed Sparse Row format>

In [180]:
# d = dict(zip(vectorizer.get_feature_names(), text.data))
# d

In [183]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(text_new, final["sentiment"], test_size=0.3, random_state=1)

In [213]:
# try logistic regression first
from sklearn.linear_model import LogisticRegression
lgr_classifier = LogisticRegression(random_state=1)
lgr_classifier.fit(x_train, y_train)
y_pred = lgr_classifier.predict(x_test)
y_pred_tr = lgr_classifier.predict(x_train)

In [214]:
print('Test accuracy', sum(y_test == y_pred)/len(y_test))
print('Train accuracy', sum(y_train == y_pred_tr)/len(y_train))

Test accuracy 0.9097777777777778
Train accuracy 0.9271428571428572


In [186]:
from sklearn.metrics import classification_report
print("Classification Report(Train)")
print(classification_report(y_train, y_pred_tr))
print("Classification Report(Test)")
print(classification_report(y_test, y_pred))

Classification Report(Train)
              precision    recall  f1-score   support

           0       0.92      0.55      0.69      3048
           1       0.93      0.99      0.96     17952

    accuracy                           0.93     21000
   macro avg       0.92      0.77      0.82     21000
weighted avg       0.93      0.93      0.92     21000

Classification Report(Test)
              precision    recall  f1-score   support

           0       0.85      0.47      0.61      1329
           1       0.91      0.99      0.95      7671

    accuracy                           0.91      9000
   macro avg       0.88      0.73      0.78      9000
weighted avg       0.91      0.91      0.90      9000



In [187]:
# Random Forests
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
y_pred_tr = classifier.predict(x_train)

In [188]:
print('Test accuracy', sum(y_test == y_pred)/len(y_test))
print('Train accuracy', sum(y_train == y_pred_tr)/len(y_train))

Test accuracy 0.9301111111111111
Train accuracy 0.9995238095238095


In [189]:
print("Classification Report(Train)")
print(classification_report(y_train, y_pred_tr))
print("Classification Report(Test)")
print(classification_report(y_test, y_pred))

Classification Report(Train)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3048
           1       1.00      1.00      1.00     17952

    accuracy                           1.00     21000
   macro avg       1.00      1.00      1.00     21000
weighted avg       1.00      1.00      1.00     21000

Classification Report(Test)
              precision    recall  f1-score   support

           0       0.91      0.58      0.71      1329
           1       0.93      0.99      0.96      7671

    accuracy                           0.93      9000
   macro avg       0.92      0.79      0.84      9000
weighted avg       0.93      0.93      0.92      9000



In [190]:
# save the model to disk
import pickle
filename = 'trained_model_rf_sentiment_analysis_10872_features.pkl'
pickle.dump(classifier, open(filename, 'wb'))

## Save the model

In [193]:
Tag_time = datetime.datetime.now().strftime("%Y-%m-%d")
time = datetime.datetime.now().strftime("%Y-%m-%d#%H:%M:%S")
tags = 'Training_version={}&Trained_date={}'.format('V1', Tag_time)

In [194]:
## arguments 

model_name = 'V1_amazon_senti-analysis_model_rf_'+time+'.pkl'
print(model_name)
path = 'ml/training_models/'

bucket = 'swire-datalake-dev-bucket'
trained_model =  classifier
key = path + model_name

print(model_name)

def save_trained_model_to_s3 (trained_model, bucket, key):
    
    """
    store the saved modelas pickle file with the tagging 
    """    
    pickle_byte_obj = pickle.dumps([trained_model]) 
    s3_client = boto3.client('s3')
    s3_resource = boto3.resource('s3')
    versioning = s3_resource.BucketVersioning(bucket)
    # check status
    print(versioning.status)
    # enable versioning
    versioning.enable()
    response = s3_client.put_object(
                                    Bucket=bucket,
                                    Body=pickle_byte_obj,
                                    Key=key,
                                    Tagging=tags
                                    )
    
    return response
save_trained_model_to_s3 (trained_model, bucket, key)

V1_amazon_senti-analysis_model_rf_2020-09-30#09:56:16.pkl
V1_amazon_senti-analysis_model_rf_2020-09-30#09:56:16.pkl
Enabled


{'ResponseMetadata': {'RequestId': '5E48DAB992A627A1',
  'HostId': 'TxdYXCDJgeE16nn2OFPjjdeoU9OHBixn8SB51AQzB3VHA0PyzGGBc35jN1g01MQ/UZXg90GGSL4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'TxdYXCDJgeE16nn2OFPjjdeoU9OHBixn8SB51AQzB3VHA0PyzGGBc35jN1g01MQ/UZXg90GGSL4=',
   'x-amz-request-id': '5E48DAB992A627A1',
   'date': 'Wed, 30 Sep 2020 09:56:19 GMT',
   'x-amz-version-id': 'yTJKFoJkvywXuygPI0DphVz7.269hMgo',
   'etag': '"d21fc5623517d6799a9a02341420b176"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'ETag': '"d21fc5623517d6799a9a02341420b176"',
 'VersionId': 'yTJKFoJkvywXuygPI0DphVz7.269hMgo'}

## save trained vector

In [196]:
## arguments 

vector_name = 'V1_tfidf_vector_amazon'+time+'.pkl'
print(model_name)
path = 'ml/training_models/'

bucket = 'swire-datalake-dev-bucket'
trained_vector =  tf_transformer
key = path + vector_name

print(model_name)

def save_trained_tfidf_vector_to_s3 (trained_vector, bucket, key):
    
    """
    store the saved modelas pickle file with the tagging 
    """    
    pickle_byte_obj = pickle.dumps([trained_model]) 
    s3_client = boto3.client('s3')
    s3_resource = boto3.resource('s3')
    versioning = s3_resource.BucketVersioning(bucket)
    # check status
    print(versioning.status)
    # enable versioning
    versioning.enable()
    response = s3_client.put_object(
                                    Bucket=bucket,
                                    Body=pickle_byte_obj,
                                    Key=key,
                                    Tagging=tags
                                    )
    
    return response
save_trained_tfidf_vector_to_s3  (trained_vector, bucket, key)

V1_amazon_senti-analysis_model_rf_2020-09-30#09:56:16.pkl
V1_amazon_senti-analysis_model_rf_2020-09-30#09:56:16.pkl
Enabled


{'ResponseMetadata': {'RequestId': 'FQDV9NCMDT7W2Z7M',
  'HostId': 'ILGlf23+wGNBcr/P2+jOhJ18iUMxCH/UzJiSRlhcdFetshvS2AMMz3NdFRodLyB7EjYplKE79dM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ILGlf23+wGNBcr/P2+jOhJ18iUMxCH/UzJiSRlhcdFetshvS2AMMz3NdFRodLyB7EjYplKE79dM=',
   'x-amz-request-id': 'FQDV9NCMDT7W2Z7M',
   'date': 'Wed, 30 Sep 2020 09:59:31 GMT',
   'x-amz-version-id': 'jZAhDWAu6t5DHtPz7VM_zj6z.ZS_yWo6',
   'etag': '"d21fc5623517d6799a9a02341420b176"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'ETag': '"d21fc5623517d6799a9a02341420b176"',
 'VersionId': 'jZAhDWAu6t5DHtPz7VM_zj6z.ZS_yWo6'}

# copy updated model to prediction model store

In [197]:
#https://stackoverflow.com/questions/44384072/move-copy-data-from-one-folder-to-another-on-aws-s3

bucket ='swire-datalake-dev-bucket'
#training 
train_path = "ml/training_models"
trained_model_file_name = "V1_amazon_senti-analysis_model_rf_2020-09-30#09:56:16.pkl"
trained_model_key  = train_path + "/" +  trained_model_file_name
#tags
tags = 'Trained_model_name == {}'.format(trained_model_file_name)
print(tags)
# prediction
pre_path = "ml/prediction_model"
prediction_model_file_name =  "amazon_sentiment_analysis_prediction_model.pkl" 
predic_model_key =  pre_path + "/" + prediction_model_file_name

Trained_model_name == V1_amazon_senti-analysis_model_rf_2020-09-30#09:56:16.pkl


In [198]:

def update_the_prediction_model (bucket,trained_model_key,predic_model_key):
    """ 
    SAve the updated trained model to prediction storage
    """
    s3 = boto3.resource('s3')
    versioning = s3.BucketVersioning(bucket)
    # check status
    print(versioning.status)
    copy_source =  {
                   'Bucket':bucket,
                   'Key': trained_model_key
                    } 
    return s3.meta.client.copy(copy_source, bucket, predic_model_key)


update_the_prediction_model (bucket,trained_model_key,predic_model_key)


Enabled


# save metadata

In [210]:
# requires
x_train = x_train
y_train =  y_train
classifier_name =classifier

In [211]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score


predictions = cross_val_predict(classifier, x_train, y_train, cv=3)
confusion_matrix(y_train, predictions)



# getting the probabilities of our predictions
y_scores = classifier_name.predict_proba(x_train)
y_scores = y_scores[:,1]



AUC_score = roc_auc_score(y_train, y_scores)

f1_score=f1_score(y_train, predictions)

Precision=precision_score(y_train, predictions)

Recall=recall_score(y_train, predictions)

Accuracy= round(classifier_name.score(x_train, y_train) * 100, 2)

cm=confusion_matrix(y_train, predictions)

r_a_score = roc_auc_score(y_train, y_scores)
print("ROC-AUC-Score:", r_a_score)

ROC-AUC-Score: 0.9999823183344563


In [212]:
evaluation_metrics={"accuracy":Accuracy,
                    "cross_val_score": np.NaN,
                   "auc_score":round(AUC_score*100,2),
                   "f1_score":round(f1_score*100,2),
                   "precision":round(Precision*100,2),
                   "recall":round(Recall*100,2),
                   "confusion_matrix": str(cm),
                    "time_stamp":datetime.datetime.now().strftime("%Y-%m-%d#%H:%M:%S") }


model_params = classifier_name.get_params()
#metadata=list(x_train.columns)

meta = {
        "model_name":"Random Forest" ,
        'model_parameters':model_params,
        #'model_features':metadata,
        "evaluation_metrics":evaluation_metrics 
       }
json_meta = json.dumps(meta)
json_meta

'{"model_name": "Random Forest", "model_parameters": {"bootstrap": true, "ccp_alpha": 0.0, "class_weight": null, "criterion": "gini", "max_depth": null, "max_features": "auto", "max_leaf_nodes": null, "max_samples": null, "min_impurity_decrease": 0.0, "min_impurity_split": null, "min_samples_leaf": 1, "min_samples_split": 2, "min_weight_fraction_leaf": 0.0, "n_estimators": 100, "n_jobs": null, "oob_score": false, "random_state": null, "verbose": 0, "warm_start": false}, "evaluation_metrics": {"accuracy": 99.95, "cross_val_score": NaN, "auc_score": 100.0, "f1_score": 95.85, "precision": 92.68, "recall": 99.25, "confusion_matrix": "[[ 1640  1408]\\n [  135 17817]]", "time_stamp": "2020-10-01#01:27:23"}}'

In [208]:
bucket ='swire-datalake-dev-bucket'
path = 'ml/metadata/nlp/amazon_sentiment_analysis/model_metadata'
file_name = 'V1_Rf_'+time+'.txt'

key = path + "/" + file_name
tags = 'Training_version={}&Trained_date={}'.format('V2', Tag_time)

def write_model_metadata_to_s3 (bucket,key,data,tags):  
    """
    store the metadata like hyperparameters, input features and 
    evaluation metrics with the tagging 
    """
    
    # Versioning 
    s3_resource = boto3.resource('s3')
    versioning = s3_resource.BucketVersioning(bucket)
    versioning.enable()
    # Put object
    s3_client = boto3.client('s3')
    response = s3_client.put_object(
                                    Bucket=bucket,
                                    Body=data,
                                    Key=key,
                                    Tagging=tags
                                    )
    
    return response

write_model_metadata_to_s3 (bucket,key,json_meta,tags)

{'ResponseMetadata': {'RequestId': '40B5E3CB3AD6890D',
  'HostId': 'N+B7uGv3nUEMzgSBafuY3swk7zJgrfEzKVfTls04eFHZgajOCBkeYRie23T+lGi0Wo7tN5AUFmw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'N+B7uGv3nUEMzgSBafuY3swk7zJgrfEzKVfTls04eFHZgajOCBkeYRie23T+lGi0Wo7tN5AUFmw=',
   'x-amz-request-id': '40B5E3CB3AD6890D',
   'date': 'Thu, 01 Oct 2020 01:09:39 GMT',
   'x-amz-version-id': 'njwavusSioyqdS2yjXd0qptQWxuRyxbp',
   'etag': '"fc25698780a0baf03c65c929fa7d30b0"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'ETag': '"fc25698780a0baf03c65c929fa7d30b0"',
 'VersionId': 'njwavusSioyqdS2yjXd0qptQWxuRyxbp'}

## Save multiple model results

In [225]:
# requires
x_train = x_train
y_train =  y_train
classifier_name = lgr_classifier #classifier
model_name ="logistic_regression" # "Random Forest" # 

In [226]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score


predictions = cross_val_predict(classifier_name, x_train, y_train, cv=3)
confusion_matrix(y_train, predictions)



# getting the probabilities of our predictions
y_scores = classifier_name.predict_proba(x_train)
y_scores = y_scores[:,1]



AUC_score = roc_auc_score(y_train, y_scores)

f1_score=f1_score(y_train, predictions)

Precision=precision_score(y_train, predictions)

Recall=recall_score(y_train, predictions)

Accuracy= round(classifier_name.score(x_train, y_train) * 100, 2)

cm=confusion_matrix(y_train, predictions)

r_a_score = roc_auc_score(y_train, y_scores)
print("ROC-AUC-Score:", r_a_score)

ROC-AUC-Score: 0.9517414713514253


In [227]:
evaluation_metrics={"accuracy":Accuracy,
                    "cross_val_score": np.NaN,
                   "auc_score":round(AUC_score*100,2),
                   "f1_score":round(f1_score*100,2),
                   "precision":round(Precision*100,2),
                   "recall":round(Recall*100,2),
                   "confusion_matrix": str(cm),
                    "time_stamp":datetime.datetime.now().strftime("%Y-%m-%d#%H:%M:%S") }


model_params = classifier_name.get_params()
#metadata=list(x_train.columns)

meta2 = {
        "model_name": model_name,
        'model_parameters':model_params,
        #'model_features':metadata,
        "evaluation_metrics":evaluation_metrics 
       }


In [234]:
meta1

{'model_name': 'Random Forest',
 'model_parameters': {'bootstrap': True,
  'ccp_alpha': 0.0,
  'class_weight': None,
  'criterion': 'gini',
  'max_depth': None,
  'max_features': 'auto',
  'max_leaf_nodes': None,
  'max_samples': None,
  'min_impurity_decrease': 0.0,
  'min_impurity_split': None,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'n_estimators': 100,
  'n_jobs': None,
  'oob_score': False,
  'random_state': None,
  'verbose': 0,
  'warm_start': False},
 'evaluation_metrics': {'accuracy': 99.95,
  'cross_val_score': nan,
  'auc_score': 100.0,
  'f1_score': 95.81,
  'precision': 92.66,
  'recall': 99.18,
  'confusion_matrix': '[[ 1638  1410]\n [  148 17804]]',
  'time_stamp': '2020-10-01#01:49:18'}}

In [236]:
# change the names for each model
#meta_name1 = model_name+"_meta" 
meta_name2 = model_name+"_meta" 
#meta2
meta={"Randomforest_meta" : meta1,
      meta_name2 : meta2
      }
meta

{'Randomforest_meta': {'model_name': 'Random Forest',
  'model_parameters': {'bootstrap': True,
   'ccp_alpha': 0.0,
   'class_weight': None,
   'criterion': 'gini',
   'max_depth': None,
   'max_features': 'auto',
   'max_leaf_nodes': None,
   'max_samples': None,
   'min_impurity_decrease': 0.0,
   'min_impurity_split': None,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0.0,
   'n_estimators': 100,
   'n_jobs': None,
   'oob_score': False,
   'random_state': None,
   'verbose': 0,
   'warm_start': False},
  'evaluation_metrics': {'accuracy': 99.95,
   'cross_val_score': nan,
   'auc_score': 100.0,
   'f1_score': 95.81,
   'precision': 92.66,
   'recall': 99.18,
   'confusion_matrix': '[[ 1638  1410]\n [  148 17804]]',
   'time_stamp': '2020-10-01#01:49:18'}},
 'logistic_regression_meta': {'model_name': 'logistic_regression',
  'model_parameters': {'C': 1.0,
   'class_weight': None,
   'dual': False,
   'fit_intercept': True,
   'intercept_scalin

In [237]:
json_meta = json.dumps(meta)
json_meta

'{"Randomforest_meta": {"model_name": "Random Forest", "model_parameters": {"bootstrap": true, "ccp_alpha": 0.0, "class_weight": null, "criterion": "gini", "max_depth": null, "max_features": "auto", "max_leaf_nodes": null, "max_samples": null, "min_impurity_decrease": 0.0, "min_impurity_split": null, "min_samples_leaf": 1, "min_samples_split": 2, "min_weight_fraction_leaf": 0.0, "n_estimators": 100, "n_jobs": null, "oob_score": false, "random_state": null, "verbose": 0, "warm_start": false}, "evaluation_metrics": {"accuracy": 99.95, "cross_val_score": NaN, "auc_score": 100.0, "f1_score": 95.81, "precision": 92.66, "recall": 99.18, "confusion_matrix": "[[ 1638  1410]\\n [  148 17804]]", "time_stamp": "2020-10-01#01:49:18"}}, "logistic_regression_meta": {"model_name": "logistic_regression", "model_parameters": {"C": 1.0, "class_weight": null, "dual": false, "fit_intercept": true, "intercept_scaling": 1, "l1_ratio": null, "max_iter": 100, "multi_class": "auto", "n_jobs": null, "penalty": 

In [238]:
bucket ='swire-datalake-dev-bucket'
path = 'ml/metadata/nlp/amazon_sentiment_analysis/multiple_model_results'
file_name = 'V1__'+time+'.txt'

key = path + "/" + file_name
tags = 'Training_version={}&Trained_date={}'.format('V1', Tag_time)

def write_model_metadata_to_s3 (bucket,key,data,tags):  
    """
    store the metadata like hyperparameters, input features and 
    evaluation metrics with the tagging 
    """
    
    # Versioning 
    s3_resource = boto3.resource('s3')
    versioning = s3_resource.BucketVersioning(bucket)
    versioning.enable()
    # Put object
    s3_client = boto3.client('s3')
    response = s3_client.put_object(
                                    Bucket=bucket,
                                    Body=data,
                                    Key=key,
                                    Tagging=tags
                                    )
    
    return response



In [239]:
write_model_metadata_to_s3 (bucket,key,json_meta,tags)

{'ResponseMetadata': {'RequestId': '3B5C832D0E3E967E',
  'HostId': 'smMh4nn+G6aTFOckTRpLHdBfJWVuOAZ3NWWqLSLtDmvNXu1CLS2Owv1KhHofLIvZoHglysuY24o=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'smMh4nn+G6aTFOckTRpLHdBfJWVuOAZ3NWWqLSLtDmvNXu1CLS2Owv1KhHofLIvZoHglysuY24o=',
   'x-amz-request-id': '3B5C832D0E3E967E',
   'date': 'Thu, 01 Oct 2020 01:56:45 GMT',
   'x-amz-version-id': 'dqnpRxCCmys0PDeEXeqU7fg1XlJk0J_C',
   'etag': '"a56abfc37bb81f3c4f75b47f9f239070"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'ETag': '"a56abfc37bb81f3c4f75b47f9f239070"',
 'VersionId': 'dqnpRxCCmys0PDeEXeqU7fg1XlJk0J_C'}

## to improve performance __-===============++++++++++++++++++++++++++++++++++++++++++++++++++=================-_______

In [83]:
# from sklearn.model_selection import GridSearchCV
# parameters = {"n_estimators": [10,50],
#              "criterion":("gini","entropy")}
# classifier = RandomForestClassifier()
# clf = GridSearchCV(classifier, parameters, cv=5)
# clf.fit(x_train, y_train)

In [84]:
#Viewing best parameters in Grid Search
best_parameter = clf.best_params_
best_accuracy = clf.best_score_ #best cros validated mean
print('Best parameter: ' + str(best_parameter))
print('Best accuracy: ' + str(best_accuracy))

Best parameter: {'criterion': 'entropy', 'n_estimators': 50}
Best accuracy: 0.9350198510502633


In [85]:
classifier = RandomForestClassifier(criterion = best_parameter["criterion"], 
                                    n_estimators = best_parameter["n_estimators"])
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
y_pred_tr = classifier.predict(x_train)
print('Test accuracy', sum(y_test == y_pred)/len(y_test))
print('Train accuracy', sum(y_train == y_pred_tr)/len(y_train))

Test accuracy 0.9276666666666666
Train accuracy 0.9996190476190476


In [86]:
print("Classification Report(Train)")
print(classification_report(y_train, y_pred_tr))
print("Classification Report(Test)")
print(classification_report(y_test, y_pred))

Classification Report(Train)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1459
           1       1.00      1.00      1.00     19541

    accuracy                           1.00     21000
   macro avg       1.00      1.00      1.00     21000
weighted avg       1.00      1.00      1.00     21000

Classification Report(Test)
              precision    recall  f1-score   support

           0       0.56      0.02      0.03       653
           1       0.93      1.00      0.96      8347

    accuracy                           0.93      9000
   macro avg       0.74      0.51      0.50      9000
weighted avg       0.90      0.93      0.89      9000



In [87]:
#! pip install keras
#! pip install tensorflow==2.2

^C


In [88]:
# save the model to disk
import pickle
filename = 'trained_model_rf_sentiment_analysis.pkl'
pickle.dump(classifier, open(filename, 'wb'))

  Using cached tensorflow-2.2.0-cp38-cp38-win_amd64.whl (459.2 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorflow_estimator-2.2.0-py2.py3-none-any.whl (454 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached absl_py-0.10.0-py3-none-any.whl (127 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached grpcio-1.32.0-cp38-cp38-win_amd64.whl (2.6 MB)
  Using cached tensorboard-2.2.2-py3-none-any.whl (3.0 MB)
Processing c:\users\vimald\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501\termcolor-1.1.0-py3-none-any.whl
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached Markdown-3.2.2-py3-none-any.whl (88 kB)
  Using cached google_auth_oauthlib-0.4.1-py2.py3-none-any.whl (18 kB)
  Using cached google_auth-1.22.0-py2.py3-none-any.whl (114 kB)
  Using cached tensorboard_plugin_wit-1.

# RNN

In [32]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, Dropout, LSTM
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [33]:
t = Tokenizer()

NameError: name 'Tokenizer' is not defined

In [ ]:
t.fit_on_texts(final["reviews.text"])

In [ ]:
max_length = max([len(s.split()) for s in final["reviews.text"] ])
max_legth=max_length #the max length is aroun 1000 character. I would keep it shorter. 

In [ ]:
vocab_size = len(t.word_index)+1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(final["reviews.text"], final["sentiment"], test_size=0.25)

In [ ]:
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_length, padding = "post",truncating = "post")
X_test = pad_sequences(X_test, maxlen=max_length, padding = "post", truncating = "post")

In [ ]:
X_train = X_train[0:28290]
y_train = y_train[0:28290]
X_test = X_test[0:9430]
y_test = y_test[0:9430]
len(y_test),len(X_test),len(X_train),len(y_train)

In [ ]:
from keras.utils import np_utils #converting to categorical
y_train = np_utils.to_categorical(y_train, num_classes=3)
y_test = np_utils.to_categorical(y_test, num_classes=3)

In [ ]:
embedding_dim = 200
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(LSTM(units = 32))
model.add(Dense(3,activation="softmax")) #since converted to categorical we will have three output nodes. softmax
                                         # assigns a probability distribution
    
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])
model.fit(X_train, y_train, batch_size=10, epochs=3)

In [ ]:
model.evaluate(x=X_test, y=y_test, batch_size=10)